<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/2_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### INSTALANDO DEPENDECIAS

In [9]:
!pip install pyspark
!pip install gcsfs
!pip install pandas

### IMPORTANDO BIBLIOTECAS, ABRINDO SPARKSESSION E CONFIGURANDO CHAVE DE SERVIÇO

In [10]:
from pyspark.sql import SparkSession
from pyspark import SQLContext
from google.cloud import storage
from pyspark.sql.types import *
from pyspark.sql.types import DateType
from pyspark.sql import Window

import pyspark
import pyspark.sql.functions as F
import os
import gcsfs
import pandas as pd

In [11]:
spark = SparkSession.builder\
.master('local')\
.appName('Projeto_Final')\
.config('spark.ui.enable', 'true')\
.config('spark.ui.port', '4050')\
.getOrCreate()

spark

serviceaccount = '/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

### IMPORTANDO, EXTRAINDO INFORMAÇÕES E ANALISANDO DATAFRAMES

In [13]:
'''https://acervolima.com/como-converter-pandas-para-pyspark-dataframe/'''

# ARQUIVO 1
file_path_1 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_despesas_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_1):
    data = pd.read_csv(file_path_1, sep=',', encoding='UTF-8', header=0)

df_1 = spark.createDataFrame(data)

# --------------------------------------------

#ARQUIVO 2
file_path_2 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencias_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_2):
    data = pd.read_json(file_path_2, encoding='UTF-8')

df_2 = spark.createDataFrame(data)

# --------------------------------------------

#ARQUIVO 3
file_path_3 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_vitimas_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_3):
    data = pd.read_json(file_path_3, encoding='UTF-8')

df_3 = spark.createDataFrame(data)

# --------------------------------------------

#ARQUIVO 4
file_path_4 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencias_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_4):
    data = pd.read_json(file_path_4, encoding='UTF-8')

df_4 = spark.createDataFrame(data)

# --------------------------------------------

#ARQUIVO 5
file_path_5 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencia_vitimas_porAnoEstado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_5):
    data = pd.read_csv(file_path_5, sep=',', encoding='UTF-8')

df_5 = spark.createDataFrame(data)

# --------------------------------------------

# ARQUIVO 6
file_path_6 = 'gs://data_lake_ingest_data/1_input/Tabela_frequencia_escolar.xlsx'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_6):
    data = pd.read_excel(file_path_6, header=0)

df_6 = spark.createDataFrame(data)

# --------------------------------------------

# ARQUIVO 7
file_path_7 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_taxa_analfabetismo_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_7):
    data = pd.read_csv(file_path_7, header=0)

df_7 = spark.createDataFrame(data)

# --------------------------------------------

df_1.printSchema()
df_1.show(5, truncate=False)
df_1.dtypes

df_2.printSchema()
df_2.show(5, truncate=False)
df_2.dtypes

df_3.printSchema()
df_3.show(5, truncate=False)
df_3.dtypes

df_4.printSchema()
df_4.show(5, truncate=False)
df_4.dtypes

df_5.printSchema()
df_5.show(5, truncate=False)
df_5.dtypes

df_6.printSchema()
df_6.show(5, truncate=False)
df_6.dtypes

df_7.printSchema()
df_7.show(5, truncate=False)
df_7.dtypes

root
 |-- UF: string (nullable = true)
 |-- Despesas2016: double (nullable = true)
 |-- Despesas2017: double (nullable = true)
 |-- Despesas2018: double (nullable = true)
 |-- Despesas2019: double (nullable = true)
 |-- Despesas2020: double (nullable = true)
 |-- Variacao%: double (nullable = true)
 |-- Previsao2021|Media: double (nullable = true)

+--------+------------------+------------+------------------+------------+------------------+---------+------------------+
|UF      |Despesas2016      |Despesas2017|Despesas2018      |Despesas2019|Despesas2020      |Variacao%|Previsao2021|Media|
+--------+------------------+------------+------------------+------------+------------------+---------+------------------+
|Acre    |498.535           |568.361     |627.351           |692.57      |493.75300000000004|8.5      |576.114           |
|Amapá   |475.32300000000004|500.227     |489.228           |606.366     |752.252           |52.3     |564.6792          |
|Amazonas|1604.7320000000002|1578.

[('Unnamed: 0', 'bigint'),
 ('UF', 'string'),
 ('Grupo_idade', 'string'),
 ('TaxaAnalfabetismo2016', 'double'),
 ('TaxaAnalfabetismo2017', 'double'),
 ('TaxaAnalfabetismo2018', 'double'),
 ('TaxaAnalfabetismo2019', 'double')]

### NORMALIZANDO DATAFRAMES


In [14]:
# DATAFRAME 1
df_1 = df_1.drop('Estimativa_2021', 'Previsao2021Media')
df_1 = df_1.drop('Estimativa_2021', 'Previsao2021Media')
df_1 = df_1.withColumnRenamed('Media Anual', 'Estimativa_Despesa_2021')
df_1 = df_1.withColumnRenamed('Previsao2021|Media', 'Previsao2021_Media')

df_1.withColumn("Despesas2017", F.round(df_1.Despesas2016.cast(FloatType()), 3))
df_1.withColumn("Despesas2017", F.round(df_1.Despesas2017.cast(FloatType()), 3))
df_1.withColumn("Despesas2018", F.round(df_1.Despesas2018.cast(FloatType()), 3))
df_1.withColumn("Despesas2019", F.round(df_1.Despesas2019.cast(FloatType()), 3))
df_1.withColumn("Despesas2020", F.round(df_1.Despesas2020.cast(FloatType()), 3))
df_1.withColumn("Previsao2021_Media", F.round(df_1.Previsao2021_Media.cast(FloatType()), 3))

df_1.show()
df_1.dtypes

+-------------------+------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                 UF|      Despesas2016|      Despesas2017|      Despesas2018|      Despesas2019|      Despesas2020|Variacao%|Previsao2021_Media|
+-------------------+------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Acre|           498.535|           568.361|           627.351|            692.57|493.75300000000004|      8.5|           576.114|
|              Amapá|475.32300000000004|           500.227|           489.228|           606.366|           752.252|     52.3|          564.6792|
|           Amazonas|1604.7320000000002|          1578.957|1862.6689999999999|           2108.46|          2101.447|     60.4|          1851.253|
|               Pará|          2551.741|          2579.916|            2903.2|          2943.448|          2966.553|     58.

[('UF', 'string'),
 ('Despesas2016', 'double'),
 ('Despesas2017', 'double'),
 ('Despesas2018', 'double'),
 ('Despesas2019', 'double'),
 ('Despesas2020', 'double'),
 ('Variacao%', 'double'),
 ('Previsao2021_Media', 'double')]

In [ ]:
# DATAFRAME 2
df_2.show(truncate=False)

In [ ]:
# DATAFRAME 3
df_3.show(truncate=False)

In [ ]:
# DATAFRAME 4
df_4.show(truncate=False)

In [ ]:
# DATAFRAME 5
df_5.show(truncate=False)

In [15]:
# DATAFRAME 6

# DROPANDO COLUNAS
df_6 = df_6.drop('15 a 17 anos - 2019', 
                 '15 a 17 anos - 2018',
                 '15 a 17 anos - 2017',
                 '15 a 17 anos - 2016',
                 '18 a 24 anos - 2019',
                 '18 a 24 anos - 2018',
                 '18 a 24 anos - 2017',
                 '18 a 24 anos - 2016',
                 '25 anos ou mais - 2019',
                 '25 anos ou mais - 2018',
                 '25 anos ou mais - 2017',
                 '25 anos ou mais - 2016')

# RENOMEANDO COLUNAS FAIXA DE IDADE
df_6 = df_6.withColumnRenamed('Grandes Regiões, Unidades da Federação e Municípios das Capitais', 'UF')
df_6 = df_6.withColumnRenamed('Media 15 a 17 anos', 'Media_Freq_15_a_17_anos')
df_6 = df_6.withColumnRenamed('média 18 a 24 anos', 'Media_Freq_18_a_24_anos')
df_6 = df_6.withColumnRenamed('média 25 anos ou mais', 'Media_Freq_25_anos_ou_mais')

# FILTRO PARA REMOVER REGIÕES E CAPITAIS
df_6 = df_6.filter((df_6.UF != "Brasil") & 
            (df_6.UF != "Norte") & 
            (df_6.UF != "Nordeste") &
            (df_6.UF != "Sudeste") & 
            (df_6.UF != "Sul") &
            (df_6.UF != "Centro-Oeste") &
            (df_6.UF != "Porto Velho") &
            (df_6.UF != "Rio Branco") &
            (df_6.UF != "Manaus") &
            (df_6.UF != "Boa Vista") &
            (df_6.UF != "Macapá") &
            (df_6.UF != "Palmas") &
            (df_6.UF != "São Luiz") &
            (df_6.UF != "Teresina") &
            (df_6.UF != "Fortaleza") &
            (df_6.UF != "Natal") &
            (df_6.UF != "joão Pessoa") &
            (df_6.UF != "Recife") &
            (df_6.UF != "Maceió") &
            (df_6.UF != "Aracaju") &
            (df_6.UF != "Salvador") &
            (df_6.UF != "Belo Horizonte") &
            (df_6.UF != "Vitória") &
            (df_6.UF != "Curitiba") &
            (df_6.UF != "Florianópolis") &
            (df_6.UF != "Porto Alegre") &
            (df_6.UF != "Campo Grande") &
            (df_6.UF != "Cuiabá") &
            (df_6.UF != "Goiânia") &
            (df_6.UF != "Brasilia"))\
            .orderBy('UF', ascending=True)

# # INCREMENTANDO TAXA DE EVASÃO
# df_6 = df_6.withColumn('Evasao_11_a_14_anos', (df_5.Freq_11_a_14_anos - 100)) 
# df_6 = df_6.withColumn('Evasao_15_a_17_anos', (df_5.Freq_11_a_14_anos - 100))
# df_6 = df_6.withColumn('Evasao_18_a_24_anos', (df_5.Freq_11_a_14_anos - 100))
# df_6 = df_6.withColumn('Evasao_25_anos_ou_mais', (df_5.Freq_11_a_14_anos - 100))

# CONVERTENDO VALORES EM PORCENTAGEM
'''https://stackoverflow.com/questions/60673912/how-to-convert-number-into-percentage'''

df_6 = df_6\
.withColumn("Media_Freq_15_a_17_anos", F.concat((F.col("Media_Freq_15_a_17_anos") * 1).cast("int"), F.lit(' %')))\
.withColumn("Media_Freq_18_a_24_anos", F.concat((F.col("Media_Freq_18_a_24_anos") * 1).cast("int"), F.lit(' %')))\
.withColumn("Media_Freq_25_anos_ou_mais", F.concat((F.col("Media_Freq_25_anos_ou_mais") * 1).cast("int"), F.lit(' %')))
# .withColumn("Media_Freq_Escolar", F.concat((F.col("Media_Freq_Escolar") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_11_a_14_anos", F.concat((F.col("Evasao_11_a_14_anos") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_15_a_17_anos", F.concat((F.col("Evasao_15_a_17_anos") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_18_a_24_anos", F.concat((F.col("Evasao_18_a_24_anos") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_25_anos_ou_mais", F.concat((F.col("Evasao_25_anos_ou_mais") * 1).cast("int"), F.lit(' %')))

# AJUSTANDO EXIBIÇÃO DO DATAFRAME
df_6 = df_6.select('UF', 
                   'Media_Freq_15_a_17_anos', 
                #    'Evasao_11_a_14_anos',
                   'Media_Freq_18_a_24_anos',
                #    'Evasao_15_a_17_anos',
                   'Media_Freq_25_anos_ou_mais',
                #    'Evasao_18_a_24_anos',
                #    'Freq_25_anos_ou_mais',
                #    'Evasao_25_anos_ou_mais',
                #    'Media_Freq_Escolar'
               )


In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
'''https://sparkbyexamples.com/pyspark/convert-pyspark-dataframe-to-pandas/'''

df_6 = df_6.toPandas()
df_6.to_csv('temp_pyspark_tabela_frequencia_escolar_normalizado.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_tabela_frequencia_escolar_normalizado.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_tabela_frequencia_escolar_normalizado.csv')

In [16]:
# DATAFRAME 7
df_7 = df_7.drop('Grupo_idade')
df_7 = df_7.drop('Unnamed: 0')

df_7 = df_7\
.withColumn("TaxaAnalfabetismo2016", F.concat((F.col("TaxaAnalfabetismo2016") * 1).cast("int"), F.lit(' %')))\
.withColumn("TaxaAnalfabetismo2017", F.concat((F.col("TaxaAnalfabetismo2017") * 1).cast("int"), F.lit(' %')))\
.withColumn("TaxaAnalfabetismo2018", F.concat((F.col("TaxaAnalfabetismo2018") * 1).cast("int"), F.lit(' %')))\
.withColumn("TaxaAnalfabetismo2019", F.concat((F.col("TaxaAnalfabetismo2019") * 1).cast("int"), F.lit(' %')))

df_7.show()

+-------------------+---------------------+---------------------+---------------------+---------------------+
|                 UF|TaxaAnalfabetismo2016|TaxaAnalfabetismo2017|TaxaAnalfabetismo2018|TaxaAnalfabetismo2019|
+-------------------+---------------------+---------------------+---------------------+---------------------+
|           Rondônia|                  6 %|                  7 %|                  6 %|                  6 %|
|               Acre|                 13 %|                 12 %|                 12 %|                 11 %|
|           Amazonas|                  6 %|                  6 %|                  5 %|                  5 %|
|            Roraima|                  6 %|                  6 %|                  6 %|                  5 %|
|               Pará|                  9 %|                  8 %|                  8 %|                  8 %|
|              Amapá|                  5 %|                  5 %|                  6 %|                  5 %|
|         

### AGREGANDO DADOS

In [ ]:
# VISUALIZAR DATAFRAMES TRATADOS

df_1.show(5, truncate=False)
df_2.show(5, truncate=False)
df_3.show(5, truncate=False)
df_4.show(5, truncate=False)
df_5.show(5, truncate=False)
df_6.show(5, truncate=False)
df_7.show(5, truncate=False)

In [ ]:
# AGREGAÇÃO 1

#AGRUPANDO VALORES DOS DATAFRAMES
df_ins2 = df_2.groupBy('UF').sum('Ocorrencias').orderBy('UF')
df_ins2.show(50, truncate=False)

df_ins3 = df_3.groupBy('UF').sum('Vitimas').orderBy('UF')
df_ins3.show(50, truncate=False)

# ------------------------------------------------------------

# JOIN 1
# AGREGA OS VALORES TOTAIS DE OCORRENCIAS E VITIMAS

df_join_1 = df_ins2.join(df_ins3, on=['UF'], how='inner')\
.select('UF', 'sum(Ocorrencias)', 'sum(Vitimas)')\
.orderBy(F.asc('UF'))

df_join_1.show(50, False)

# ------------------------------------------------------------

# JOIN 2
# AGREGA OS VALORES TOTAIS DE OCORRENCIAS, VITIMAS, MEDIA DE FREQ ESCOLAR

df_join_1 = df_join_1.join(df_6, on=['UF'], how='inner')\
.select('UF', 'sum(Ocorrencias)', 'sum(Vitimas)', 'Media_Freq_25_anos_ou_mais')\
.orderBy('UF', ascending=True)

df_join_1.show(50, False)

# ------------------------------------------------------------

# JOIN 3
# AGREGA OS VALORES TOTAIS DE DESPESAS GOVERNAMENTAIS, OCORRENCIAS, VITIMAS, MEDIA DE FREQ. ESCOLAR

df_join_1 = df_join_1.join(df_1, on=['UF'], how='inner')\
.select('UF', 'Despesas2020', 'sum(Ocorrencias)', 'sum(Vitimas)', 'Media_Freq_25_anos_ou_mais')\
.orderBy(F.asc('UF'))

df_join_1\
.orderBy("Despesas2020", ascending=False)\
.show(50, False)

df_join_3.dtypes

# ------------------------------------------------------------

df_join_1 = df_join_1.toPandas()
df_join_1.to_csv('temp_pyspark_agg_1_General_Data.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_1_General_Data.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_1_General_Data.csv')

In [ ]:
# AGREGAÇÃO 2

# JOIN 1
# AGREGA OS VALORES TOTAIS DE OCORRENCIAS E VITIMAS POR ESTADO 

# ------------------------------------------------------------

df_join_1 = df_6.join(df_7, on=['UF'], how='inner')\
.select('UF', 'Media_Freq_15_a_17_anos', 'Media_Freq_18_a_24_anos', 'Media_Freq_25_anos_ou_mais', 
        'TaxaAnalfabetismo2016', 'TaxaAnalfabetismo2017', 'TaxaAnalfabetismo2018', 'TaxaAnalfabetismo2019')\
.orderBy(F.asc('UF'))\

# ------------------------------------------------------------

df_join_1 = df_join_1.toPandas()
df_join_1.to_csv('temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv')

In [ ]:
# AGREGAÇÃO 3

# AGRUPANDO VALORES DOS DATAFRAMES
df_agg_1 = df_2.filter(df_2.Ano == 2020)\
.groupBy('UF').sum('Ocorrencias').alias('Ocorrencias_2020').orderBy('UF')

df_agg_1.show()

# ------------------------------------------------------------

df_agg_2 = df_3.filter(df_3.Ano == 2020)\
.groupBy('UF').sum('Vitimas').alias('Vitimas_2020').orderBy('UF')

df_agg_2.show()

# ------------------------------------------------------------

# AGREGANDO OS VALORES PELO ANO DE 2020

df_join_1 = df_agg_1.join(df_agg_2, on=['UF'], how='inner')\
.select('UF', 'sum(Ocorrencias)', 'sum(Vitimas)')\
.orderBy('UF', ascending=True)

df_join_1.show()

# ------------------------------------------------------------

df_join_2 = df_1.select('UF', 'Despesas2020')

df_join_1 = df_join_1.join(df_join_2, on=['UF'], how='inner')\
.select('UF', 
        'sum(Ocorrencias)', 
        'sum(Vitimas)', 
        'Despesas2020')

df_join_1 = df_join_1\
.withColumnRenamed('sum(Ocorrencias)', 'Ocorrencias_2020')\
.withColumnRenamed('sum(Vitimas)', 'Vitimas_2020')\
.withColumnRenamed('Despesas2020', 'Despesas_Gov_2020')\

df_join_1.show()

# ------------------------------------------------------------

df_join_1 = df_join_1.toPandas()
df_join_1.to_csv('temp_pyspark_agg_3_general_data_2020.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_3_general_data_2020.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_3_general_data_2020.csv')

### TEST FUNCTIONS

In [17]:
# RANKING

w0 = Window.partitionBy(F.col('TipoCrime')).orderBy('UF')

df_2.withColumn('Ranking', F.row_number().over(w0))\
.select('Ranking', 'UF', 'TipoCrime')\
.show()

+-------+----+---------+
|Ranking|  UF|TipoCrime|
+-------+----+---------+
|      1|Acre|  Estupro|
|      2|Acre|  Estupro|
|      3|Acre|  Estupro|
|      4|Acre|  Estupro|
|      5|Acre|  Estupro|
|      6|Acre|  Estupro|
|      7|Acre|  Estupro|
|      8|Acre|  Estupro|
|      9|Acre|  Estupro|
|     10|Acre|  Estupro|
|     11|Acre|  Estupro|
|     12|Acre|  Estupro|
|     13|Acre|  Estupro|
|     14|Acre|  Estupro|
|     15|Acre|  Estupro|
|     16|Acre|  Estupro|
|     17|Acre|  Estupro|
|     18|Acre|  Estupro|
|     19|Acre|  Estupro|
|     20|Acre|  Estupro|
+-------+----+---------+
only showing top 20 rows



In [18]:
# STRUCTTYPE E UNION BY NAME

esquema = (StructType([
        StructField('UF', StringType(), True),
        StructField('TipoCrime', StringType(), True),
        StructField('Ano', IntegerType(), True),
        StructField('Mes', IntegerType(), True),
        StructField('Ocorrencias', IntegerType(), True)
    ])
)

df = spark.createDataFrame(data='', schema=esquema)
df.printSchema()

df.unionByName(df_2).show()

root
 |-- UF: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Ocorrencias: integer (nullable = true)

+----+--------------------+----+---------+-----------+
|  UF|           TipoCrime| Ano|      Mes|Ocorrencias|
+----+--------------------+----+---------+-----------+
|Acre|             Estupro|2021|  janeiro|         39|
|Acre|    Furto de veículo|2021|  janeiro|         55|
|Acre|    Homicídio doloso|2021|  janeiro|         14|
|Acre|Lesão corporal se...|2021|  janeiro|          0|
|Acre|Roubo a instituiç...|2021|  janeiro|          0|
|Acre|      Roubo de carga|2021|  janeiro|          0|
|Acre|    Roubo de veículo|2021|  janeiro|         99|
|Acre|Roubo seguido de ...|2021|  janeiro|          1|
|Acre|Tentativa de homi...|2021|  janeiro|         10|
|Acre|             Estupro|2021|fevereiro|         23|
|Acre|    Furto de veículo|2021|fevereiro|         49|
|Acre|    Homicídio doloso|2021|f

In [19]:
# HEADER COM LISTAS

file_path_1 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_despesas_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_1):
    data = pd.read_csv(file_path_1,
                       sep=',', 
                       encoding='UTF-8')

esquema=['UF', 'Despesas2016', 'Despesas2017', 'Despesas2018', 'Despesas2019', 'Despesas2020', 'Variacao%', 'Previsao2021_Media']

df_1 = spark.createDataFrame(data, schema=esquema)

# --------------------------------------------
    
df_1.show()

+-------------------+------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                 UF|      Despesas2016|      Despesas2017|      Despesas2018|      Despesas2019|      Despesas2020|Variacao%|Previsao2021_Media|
+-------------------+------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Acre|           498.535|           568.361|           627.351|            692.57|493.75300000000004|      8.5|           576.114|
|              Amapá|475.32300000000004|           500.227|           489.228|           606.366|           752.252|     52.3|          564.6792|
|           Amazonas|1604.7320000000002|          1578.957|1862.6689999999999|           2108.46|          2101.447|     60.4|          1851.253|
|               Pará|          2551.741|          2579.916|            2903.2|          2943.448|          2966.553|     58.